In [1]:
#Libraraynya

import nltk
nltk.download('punkt')
nltk.download('wordnet')
from nltk.stem import WordNetLemmatizer
lemmatizer = WordNetLemmatizer()
import json
import pickle

import numpy as np
from tensorflow.keras import Sequential
from tensorflow.keras.layers import Conv2D, Flatten, Dense, Dropout
from tensorflow.keras.optimizers import SGD
import random



[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\Lenovo\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\Lenovo\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


In [3]:
#Data Preprocessing
#Medefinisikan word, Classes, Document kedalam sebauh array 
#Lalu melakukan pengolaan data dari data yang sudah kita buat yaitu data.json
words=[]
classes = []
documents = []
ignore_words = ['?', '!']
data_file = open('data.json').read()
intents = json.loads(data_file)

In [4]:
#datanya itu terdiri dari tag, Patterns, Response (model pembicaraan atau percakapan)
intents

{'intents': [{'tag': 'siapa',
   'patterns': ['Siapa kamu?', 'Kamu siapa?'],
   'responses': ['Saya Hanya sebuah bot RumahTBC, Ayo mulai dengan pertanyaanmu..'],
   'context_filter': 'rentalday'},
  {'tag': 'Hai',
   'patterns': ['Hello', 'Hi', 'Hey', 'Hallo'],
   'responses': ['Hallo! guys',
    'Hai, Semuanya',
    'Hallo, Hallo Siapa nama mu?'],
   'context': ['']},
  {'tag': 'noanswer',
   'patterns': [''],
   'responses': ['Maaf pertanyaan anda tidak sesuai, Tanyakan yang lain ...'],
   'context': ['']},
  {'tag': 'Nama',
   'patterns': ['Nama Saya', 'Panggilan Saya'],
   'responses': ['Senang Bertemu dengan mu', 'Namaku '],
   'context': ['']},
  {'tag': 'bantuan',
   'patterns': ['saya butuh bantuan',
    'bisakah kamu bantu saya',
    'kamu bisa bantu saya?'],
   'responses': ['ya, saya bisa bantu anda', 'ya, apa itu?'],
   'context': ['']},
  {'tag': 'anemia_definition',
   'patterns': ['Apa itu anemia?', 'Definisi anemia', 'Pengertian anemia'],
   'responses': ['Anemia adalah

In [5]:
#Melakukan Orgenazing membuat sekelompok kata
for intent in intents['intents']:
    for pattern in intent['patterns']:

        # mengambil setiap kata dan tokenize
        w = nltk.word_tokenize(pattern) #port yg ada didalam sebuah pattern itu nanti akan di orgenize 
        words.extend(w)
        # akan ditambah ke dalam document
        documents.append((w, intent['tag']))

        # Menambah class ke dlm class yg sudah kita buat.
        if intent['tag'] not in classes:
            classes.append(intent['tag'])

In [6]:
words #setiap kata yg sudah di rangkai didalam intenst.json

['Siapa',
 'kamu',
 '?',
 'Kamu',
 'siapa',
 '?',
 'Hello',
 'Hi',
 'Hey',
 'Hallo',
 'Nama',
 'Saya',
 'Panggilan',
 'Saya',
 'saya',
 'butuh',
 'bantuan',
 'bisakah',
 'kamu',
 'bantu',
 'saya',
 'kamu',
 'bisa',
 'bantu',
 'saya',
 '?',
 'Apa',
 'itu',
 'anemia',
 '?',
 'Definisi',
 'anemia',
 'Pengertian',
 'anemia',
 'Apa',
 'jenis-jenis',
 'anemia',
 '?',
 'Macam-macam',
 'anemia',
 'Bagaimana',
 'klasifikasi',
 'anemia',
 '?',
 'Apa',
 'saja',
 'gejala',
 'anemia',
 '?',
 'Bagaimana',
 'cara',
 'mengenali',
 'gejala',
 'anemia',
 '?',
 'Tanda-tanda',
 'anemia',
 'Apa',
 'penyebab',
 'anemia',
 '?',
 'Apa',
 'yang',
 'bisa',
 'menyebabkan',
 'seseorang',
 'mengalami',
 'anemia',
 '?',
 'Faktor',
 'penyebab',
 'anemia',
 'Bagaimana',
 'anemia',
 'didiagnosis',
 '?',
 'Prosedur',
 'diagnosa',
 'anemia',
 'Apa',
 'yang',
 'dilakukan',
 'dokter',
 'untuk',
 'mengidentifikasi',
 'anemia',
 '?',
 'Bagaimana',
 'pengobatan',
 'anemia',
 '?',
 'Apa',
 'jenis',
 'pengobatan',
 'yang',
 'd

In [7]:
documents #ini sudah melalui td tahan orgenize 

[(['Siapa', 'kamu', '?'], 'siapa'),
 (['Kamu', 'siapa', '?'], 'siapa'),
 (['Hello'], 'Hai'),
 (['Hi'], 'Hai'),
 (['Hey'], 'Hai'),
 (['Hallo'], 'Hai'),
 ([], 'noanswer'),
 (['Nama', 'Saya'], 'Nama'),
 (['Panggilan', 'Saya'], 'Nama'),
 (['saya', 'butuh', 'bantuan'], 'bantuan'),
 (['bisakah', 'kamu', 'bantu', 'saya'], 'bantuan'),
 (['kamu', 'bisa', 'bantu', 'saya', '?'], 'bantuan'),
 (['Apa', 'itu', 'anemia', '?'], 'anemia_definition'),
 (['Definisi', 'anemia'], 'anemia_definition'),
 (['Pengertian', 'anemia'], 'anemia_definition'),
 (['Apa', 'jenis-jenis', 'anemia', '?'], 'anemia_types'),
 (['Macam-macam', 'anemia'], 'anemia_types'),
 (['Bagaimana', 'klasifikasi', 'anemia', '?'], 'anemia_types'),
 (['Apa', 'saja', 'gejala', 'anemia', '?'], 'anemia_symptoms'),
 (['Bagaimana', 'cara', 'mengenali', 'gejala', 'anemia', '?'],
  'anemia_symptoms'),
 (['Tanda-tanda', 'anemia'], 'anemia_symptoms'),
 (['Apa', 'penyebab', 'anemia', '?'], 'anemia_causes'),
 (['Apa',
   'yang',
   'bisa',
   'menyeb

In [8]:
#Melakukan Lemmatizing, melakuka pengelompokan kata dlm dokumen/class yg udh dibuat
words = [lemmatizer.lemmatize(w.lower()) for w in words if w not in ignore_words]
words = sorted(list(set(words)))

classes = sorted(list(set(classes)))

print (len(documents), "documents")

print (len(classes), "classes", classes)

print (len(words), "unique lemmatized words", words)


pickle.dump(words,open('words.pkl','wb'))
pickle.dump(classes,open('classes.pkl','wb')) #menhasilkan 32 document dan 12 classes

48 documents
17 classes ['Hai', 'Nama', 'anemia_causes', 'anemia_complications', 'anemia_definition', 'anemia_diagnosis', 'anemia_impact', 'anemia_management', 'anemia_prevention', 'anemia_risk_factors', 'anemia_support', 'anemia_symptoms', 'anemia_treatment', 'anemia_types', 'bantuan', 'noanswer', 'siapa']
82 unique lemmatized words ['ada', 'akibat', 'anemia', 'apa', 'apakah', 'bagaimana', 'bantu', 'bantuan', 'berisiko', 'bisa', 'bisakah', 'butuh', 'cara', 'dampak', 'dapat', 'definisi', 'diagnosa', 'didiagnosis', 'digunakan', 'dilakukan', 'diri', 'dokter', 'dukungan', 'efek', 'faktor', 'faktor-faktor', 'gejala', 'hallo', 'hello', 'hey', 'hi', 'itu', 'jenis', 'jenis-jenis', 'jika', 'kamu', 'kelompok', 'kesehatan', 'klasifikasi', 'komplikasi', 'konsekuensi', 'lakukan', 'langkah-langkah', 'macam-macam', 'masalah', 'memengaruhi', 'mencegah', 'mendapatkan', 'mengalami', 'mengatasi', 'mengelola', 'mengenali', 'mengidap', 'mengidentifikasi', 'meningkatkan', 'menyebabkan', 'merawat', 'mungkin

In [9]:
classes

['Hai',
 'Nama',
 'anemia_causes',
 'anemia_complications',
 'anemia_definition',
 'anemia_diagnosis',
 'anemia_impact',
 'anemia_management',
 'anemia_prevention',
 'anemia_risk_factors',
 'anemia_support',
 'anemia_symptoms',
 'anemia_treatment',
 'anemia_types',
 'bantuan',
 'noanswer',
 'siapa']

In [11]:
#Membuat Deep Learning Model
#membuat training data dan testing data 

# initializing training data
training = [] #menginisialisaikan data training ygakan diiskan apa 
output_empty = [0] * len(classes) #output sbg outputnya nanti yg kita buat.
for doc in documents:
    
    # initializing bag sbg tempat plemparan katanya/words
    bag = []
    
    # list of tokenized words for the pattern (tempat penumpulan kata pattrens)
    pattern_words = doc[0] 
    
    #lemmatize setiap kata - buat kata dasar, dalam upaya untuk mewakili kata-kata terkait
    pattern_words = [lemmatizer.lemmatize(word.lower()) for word in pattern_words] 
    
    #jika bag of wordnya sesai dg yg ada di pattrens maka akan ditambah 1 kedalam bag klo gaa maka akan ttp 0 
    for w in words:
        bag.append(1) if w in pattern_words else bag.append(0)

    # output adalah '0' untuk setiap tag dan '1' untuk tag saat ini (untuk setiap pola)
    output_row = list(output_empty) #terdiri dr list output empety yg sudh kita buat
    output_row[classes.index(doc[1])] = 1 #ini akan ditambahkan ke dataset kita yg berisikan parameter bag  

    training.append([bag, output_row])
# shuffle our features and turn into np.array
random.shuffle(training)
# training = np.array(training) #menggunahnya ke numpy

# # create train and test lists. X - patterns, Y - intents
# train_x = list(training[:,0])
# train_y = list(training[:,1])
# print("Training data created") #Membuat Traing datanya

# Convert list of tuples into a tuple of lists
train_x, train_y = zip(*training)

# Convert to numpy arrays
train_x = np.array(train_x)
train_y = np.array(train_y)

print("Training data created")

Training data created


In [17]:
import numpy as np
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout
from tensorflow.keras.optimizers.legacy import SGD

# Asumsikan train_x dan train_y sudah tersedia sebagai numpy array
# train_x = ...
# train_y = ...

# Membuat model - 3 lapisan. Lapisan pertama dengan 128 neuron, lapisan kedua dengan 64 neuron,
# dan lapisan output dengan jumlah neuron sesuai jumlah intents untuk memprediksi output intent dengan softmax
model = Sequential()
model.add(Dense(128, input_shape=(len(train_x[0]),), activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(64, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(len(train_y[0]), activation='softmax'))

# Mengkompilasi model. Stochastic gradient descent dengan Nesterov accelerated gradient memberikan hasil yang baik untuk model ini
sgd = SGD(learning_rate=0.01, momentum=0.9, nesterov=True)
model.compile(loss='categorical_crossentropy', optimizer=sgd, metrics=['accuracy'])

# Melatih dan menyimpan model
hist = model.fit(np.array(train_x), np.array(train_y), epochs=200, batch_size=5, verbose=1)
model.save('chatbot_model.h5')

print("Model created")


Epoch 1/200
10/10 [==============================] - 0s 904us/step - loss: 2.8622 - accuracy: 0.0000e+00
Epoch 2/200
10/10 [==============================] - 0s 956us/step - loss: 2.8236 - accuracy: 0.0833
Epoch 3/200
10/10 [==============================] - 0s 2ms/step - loss: 2.7713 - accuracy: 0.1458
Epoch 4/200
10/10 [==============================] - 0s 893us/step - loss: 2.7902 - accuracy: 0.1250
Epoch 5/200
10/10 [==============================] - 0s 2ms/step - loss: 2.6637 - accuracy: 0.1458
Epoch 6/200
10/10 [==============================] - 0s 1ms/step - loss: 2.6147 - accuracy: 0.2292
Epoch 7/200
10/10 [==============================] - 0s 954us/step - loss: 2.5928 - accuracy: 0.1875
Epoch 8/200
10/10 [==============================] - 0s 2ms/step - loss: 2.5139 - accuracy: 0.2917
Epoch 9/200
10/10 [==============================] - 0s 1ms/step - loss: 2.3525 - accuracy: 0.4167
Epoch 10/200
10/10 [==============================] - 0s 2ms/step - loss: 2.3231 - accuracy: 0.35

c:\Users\Lenovo\AppData\Local\Programs\Python\Python311\Lib\site-packages\keras\src\engine\training.py:3000: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


In [19]:
from tensorflow.keras.models import load_model
model = load_model('chatbot_model.h5')
import json
import random
intents = json.loads(open('data.json').read())
words = pickle.load(open('words.pkl','rb'))
classes = pickle.load(open('classes.pkl','rb'))

In [20]:
#Pembuatan Fungsi yg akan digunakan dlm sebuah Chtbo
def clean_up_sentence(sentence): #1 utk memebrsihkan kaliamt yg diimput sebelummnya
    sentence_words = nltk.word_tokenize(sentence)
    sentence_words = [lemmatizer.lemmatize(word.lower()) for word in sentence_words]
    return sentence_words

# return bag of words array: 0 or 1 for each word in the bag that exists in the sentence

def bow(sentence, words, show_details=True): #2 menampung semua kalimat yg telah dibersihkan dam memuat kata yg dpt mempredisi class berdasarkan model yg udh dibuat
    # tokenize the pattern
    sentence_words = clean_up_sentence(sentence)
    # bag of words - matrix of N words, vocabulary matrix
    bag = [0]*len(words)
    for s in sentence_words:
        for i,w in enumerate(words):
            if w == s:
                # assign 1 if current word is in the vocabulary position
                bag[i] = 1
                if show_details:
                    print ("found in bag: %s" % w)
    return(np.array(bag))

def predict_class(sentence, model):

    # filter out predictions below a threshold
    p = bow(sentence, words,show_details=False)
    res = model.predict(np.array([p]))[0]
    error = 0.25
    results = [[i,r] for i,r in enumerate(res) if r>error]

    # sort by strength of probability
    results.sort(key=lambda x: x[1], reverse=True)
    return_list = []

    for r in results:
        return_list.append({"intent": classes[r[0]], "probability": str(r[1])})
    return return_list

def getResponse(ints, intents_json): #4 membuat list dr output yg mengecek kecocokan probabilitas yg tertinggi
    tag = ints[0]['intent']
    list_of_intents = intents_json['intents']
    for i in list_of_intents:
        if(i['tag']== tag):
            result = random.choice(i['responses'])
            break
    return result

def chatbot_response(text):
    ints = predict_class(text, model)
    res = getResponse(ints, intents)
    return res

In [21]:
#Membangun GUI dengan tkinter 
#libraryi ini menghasilkan sebuah grafik dr chtboy yg dibangun
import tkinter
from tkinter import *

# berisi class
def send(): #pengatur pesan yg kita kirim maka nanti ada responnya gtu 
    msg = EntryBox.get("1.0",'end-1c').strip()
    EntryBox.delete("0.0",END)

    if msg != '':
        ChatLog.config(state=NORMAL)
        ChatLog.insert(END, "You: " + msg + '\n\n')
        ChatLog.config(foreground="#442265", font=("Verdana", 12 ))

        res = chatbot_response(msg)
        ChatLog.insert(END, "Bot: " + res + '\n\n')

        ChatLog.config(state=DISABLED)
        ChatLog.yview(END)


base = Tk()
base.title("RumahTBC")
base.geometry("400x500")
base.resizable(width=FALSE, height=FALSE)

#Tampilan layar
ChatLog = Text(base, bd=0, bg="white", height="8", width="50", font="Arial",)

ChatLog.config(state=DISABLED)

#scroll bar utuk layar chtbotnya
scrollbar = Scrollbar(base, command=ChatLog.yview, cursor="heart")
ChatLog['yscrollcommand'] = scrollbar.set

#Tombol pengiriman pesan
SendButton = Button(base, font=("Verdana",12,'bold'), text="Kirim", width="12", height=5,
                    bd=0, bg="#32de97", activebackground="#3c9d9b",fg='#ffffff',
                    command= send )

#Tempat untuk megirimkan pesan
EntryBox = Text(base, bd=0, bg="white",width="29", height="5", font="Arial")
#EntryBox.bind("<Return>", send)


#Tampilan untuk bagian layarnya
scrollbar.place(x=376,y=6, height=386)
ChatLog.place(x=6,y=6, height=386, width=370)
EntryBox.place(x=128, y=401, height=90, width=265)
SendButton.place(x=6, y=401, height=90)

base.mainloop()

1/1 [==============================] - 0s 25ms/step
